# PART 2

## Problem （unicode1）: Understanding Unicode （1 point）

```python
>>> chr(0)
'\x00'
>>> print(chr(0))

>>> "this is a test" + chr(0) + "string"
'this is a test\x00string'
>>> print("this is a test" + chr(0) + "string")
this is a teststring
>>>
```

## Problem(unicode2): Unicode Encodings(3 points)

### (a)

- UTF-8 为变长编码，对 ASCII 字符（文本中常见）仅用 1 字节，存储和处理更高效；
- UTF-8 兼容 ASCII，在多语言文本中适应性更强
- 而 UTF-16/32 对 ASCII 文本冗余度高（UTF-16 至少 2 字节，UTF-32 固定 4 字节）。

### (b)

输入 日语或中文即可

### (c)

指的是不符合 UTF-8 规则的


## 2.4 BPE Tokenizer Training



- Vocabulary initialization
- pre-tokenization
- BPE merge
- Special tokens


### Problem: BPE Tokenizer Training (15 points)

这里主要问题在于弄清楚类型都是什么，bytes如何操作

- initial_bytes
  - bytes([i])和 bytes(i)的差异！
  - bytes(i)是i个\0拼起来

- _pre_tokenize
  - 他的任务是，拆分出初步的pretokenize的结果
  - 注意：按照special tokens 拆分开，不能跨越special token进行merge
  - 返回pre_tokens，内容为{tuple(byte1, byte2, ... , bytek) : count}
- train
  - 重点是merge部分的算法，这里是主要时耗
- 为什么采取分离的方式构建vocab?
  - 这样只需要维护`merged`就可以了


- 2025-10-20最新战报：最大堆修正失败，寄
- 2025-10-28 重整旗鼓 over
- 2025-10-29 64G机器还是OOM ToT

- 2025-10-30 打算今天根据一个blog 抄他更改的思路自己实现最后的工业级别的结果

- TODO: `Optionally (this could be a large time-investment),you can implement the key parts of your training method using some systems language, for instanceC++ (considercppyyfor this) or Rust (using PyO3)`

这里是我加速bpe_training的流程思路之类的
> 算法的优化，数据结构的优化，并行(openmp)优化，cython优化，用c++实现关键代码和c++库的cython集成

- 用yield流式读取chunk，一点一点读取
- 为了利用多个CPU进行并行计算，最常见的就是使用多线程。但是由于python的GIL，python的多线程(threading)只能实现并发(concurrency)而不是并行(parallel)，python的多线程只适合I/O密集型任务，而不适合CPU密集型任务。对于CPU密集型任务，比较合适的是使用多进程(multiprocessing)。
- 另外，我们的计算需要分成很多步骤，因此使用队列和生产者/消费者模式来实现是比较清晰的。具体来说我们的流程如下：
  - 文档切割 利用文档分隔特殊字符串<|endoftext|>把原始文本切分成多个chunk，每个chunk包含一个或多个文档。结果放到chunk_queue。
  - 正则匹配和词频统计 把chunk用<|endoftext|>切分成文档，再把文档切分成词，然后统计词频。处理的结果放到counter_queue。
  - 词频合并 从counter_queue取出chunk的词频统计结果，然后进行合并，结果放到merged_queue。
  - 最终合并 主进程把merged_queue里的统计结果进行合并。
- 遇到类似的问题，采用 8-2配置用时40s，采用16-2配置用时30s，采用16-4配置用时 30s


### Problem （train_bpe_tinystories）: BPE Training on TinyStories （2 points）

- 2025-10-29 改了下 pre-tokenize, 防止一下全放到内存里给干炸了
- 跑完了，爽


### Problem （train_bpe_expts_owt）: BPE Training on OpenWeb Text （2 points）
```
 uv run cs336_basics/train_bpe.py
chunk_queu: 0, counter_queue: 0, merged_queue: 0
chunk_queu: 36559, counter_queue: 3, merged_queue: 0
chunk_queu: 67665, counter_queue: 1, merged_queue: 0
chunk_queu: 97751, counter_queue: 1, merged_queue: 0
chunk_queu: 126147, counter_queue: 2, merged_queue: 0
chunk_queu: 153798, counter_queue: 3, merged_queue: 0
chunk_queu: 145248, counter_queue: 0, merged_queue: 0
chunk_queu: 132496, counter_queue: 1, merged_queue: 0
chunk_queu: 119565, counter_queue: 4, merged_queue: 0
chunk_queu: 106843, counter_queue: 4, merged_queue: 0
chunk_queu: 95257, counter_queue: 0, merged_queue: 0
chunk_queu: 82290, counter_queue: 2, merged_queue: 0
chunk_queu: 69311, counter_queue: 3, merged_queue: 0
chunk_queu: 56490, counter_queue: 2, merged_queue: 0
chunk_queu: 43478, counter_queue: 0, merged_queue: 0
chunk_queu: 30519, counter_queue: 0, merged_queue: 0
chunk_queu: 17555, counter_queue: 1, merged_queue: 0
chunk_queu: 4603, counter_queue: 3, merged_queue: 0
chunk_queu: 0, counter_queue: 0, merged_queue: 0
_pretokenize_and_count takes: 190.0569782579987
merge 1000: 231.10991237199778
merge 2000: 371.4132240429972
merge 3000: 575.2559400539976
merge 4000: 836.2143881919983
merge 5000: 1154.4111030869972
merge 6000: 1515.6526871149981
merge 7000: 1920.8361723350026
merge 8000: 2374.731511883001
merge 9000: 2907.8827909429965
merge time: 3333.8703679980026
Training completed in 3552.53 seconds.
Vocab size: 10000
Longest token: b'----------------' (length=16)
```

## 2.6

### Problem （tokenizer）:Implementing the tokenizer （15 points）

- 2025-10-30 差一个 Memory 限制没有过

### Problem （tokenizer_experiments）:Experiments with tokenizers （4 points）

TinyStories 的 tokenizer 在压缩效率（更低的 bytes/token）和处理速度（更高的 MB/s）上均显著优于 OpenWebText 的 tokenizer，且跨数据集时性能依然稳定。这可能与两个数据集的特性有关：TinyStories 多为短文本、简单词汇，其 tokenizer 可能更侧重 “紧凑编码”；而 OpenWebText 文本更复杂（长文本、多样词汇），tokenizer 可能为了覆盖更多复杂模式而牺牲了部分效率。
如果你的需求是高效处理大规模文本（尤其是追求速度和压缩率），TinyStories 的 tokenizer 会是更优选择。


# PART 3

## 3.3

这里在教学 `einops`

## 3.4

本次作业在数学表示上要求使用列向量 ($y = Wx$)，但在实际代码实现中，您应该回归 PyTorch/NumPy 的行向量惯例 ($y = xW^T$) 来利用其内存优化。

- 实现了 linear 和 embedding

## 3.5
> 2025-10-31 有意义的一天
> 可能重要原因之一是在GenAI实现过 MHA和RoPE和GQA

- Root Mean Square Layer Normalization([ ] 为什么work，当时课上也没有说？还是我忘记了)
- SiLU
- SwiGLU
- RoPE
- attention
- MHA

- TransformerBlock
- TransformerLM

## 3.5 transformer resource accounting


### (A)
2127057600 个 fp16 可训练

8G左右

### (B)

RJTransformerLM模型FLOPs分析：
  单块RMSNorm（注意力前）：6.55 MFLOPs
  单块注意力总：27.69 GFLOPs
  单块RMSNorm（FFN前）：6.55 MFLOPs
  单块FFN总：62.93 GFLOPs
  单块总FLOPs：90.63 GFLOPs
  所有层总FLOPs：4.35 TFLOPs
  输出层总FLOPs：164.69 GFLOPs
  模型总FLOPs：4.52 TFLOPs

### (C)
单块FFN总：62.93 GFLOPs
单块注意力总：27.69 GFLOPs


### (D)

---
Small
282472704
模型FLOPs分析：
  单块RMSNorm（注意力前）：3.15 MFLOPs
  单块注意力总：8.06 GFLOPs
  单块RMSNorm（FFN前）：3.15 MFLOPs
  单块FFN总：30.21 GFLOPs
  单块总FLOPs：38.28 GFLOPs
  所有层总FLOPs：459.31 GFLOPs
  输出层总FLOPs：79.05 GFLOPs
  模型总FLOPs：538.36 GFLOPs

---
Medium
675499008
模型FLOPs分析：
  单块RMSNorm（注意力前）：4.19 MFLOPs
  单块注意力总：12.89 GFLOPs
  单块RMSNorm（FFN前）：4.19 MFLOPs
  单块FFN总：40.28 GFLOPs
  单块总FLOPs：53.18 GFLOPs
  所有层总FLOPs：1.28 TFLOPs
  输出层总FLOPs：105.40 GFLOPs
  模型总FLOPs：1.38 TFLOPs

---
Large
1249416960
模型FLOPs分析：
  单块RMSNorm（注意力前）：5.24 MFLOPs
  单块注意力总：18.80 GFLOPs
  单块RMSNorm（FFN前）：5.24 MFLOPs
  单块FFN总：50.34 GFLOPs
  单块总FLOPs：69.15 GFLOPs
  所有层总FLOPs：2.49 TFLOPs
  输出层总FLOPs：131.75 GFLOPs
  模型总FLOPs：2.62 TFLOPs

### (E)


2127057600
模型FLOPs分析：
  单块RMSNorm（注意力前）：104.86 MFLOPs
  单块注意力总：2.05 TFLOPs
  单块RMSNorm（FFN前）：104.86 MFLOPs
  单块FFN总：1.01 TFLOPs
  单块总FLOPs：3.06 TFLOPs
  所有层总FLOPs：146.92 TFLOPs
  输出层总FLOPs：2.64 TFLOPs
  模型总FLOPs：149.55 TFLOPs

我勒个平方级






In [7]:
import torch
from cs336_basics.my_module import *

# 格式化输出（转换为易读单位）
def format_flops(value):
    for unit in ['FLOPs', 'kFLOPs', 'MFLOPs', 'GFLOPs', 'TFLOPs', 'PFLOPs']:
        if value < 1000:
            return f"{value:.2f} {unit}"
        value /= 1000
    return f"{value:.2f} PFLOPs"

def count_trainable_params(model):
    """
    统计模型中可训练参数的总数量
    
    参数:
        model: PyTorch模型实例
        
    返回:
        int: 可训练参数的总数量
    """
    # 遍历模型所有参数，筛选出requires_grad=True的参数，累加每个参数的元素数量
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def calcu(
    vocab_size: int,
    context_length: int,
    num_layers: int,
    d_model: int,
    num_heads: int,
    d_ff: int
):
    
    model = RJTransformerLM(
        vocab_size=vocab_size,
        context_length=context_length,
        num_layers=num_layers,
        d_model=d_model,
        num_heads=num_heads,
        d_ff=d_ff,
        rope_theta=10000
    )
    print(count_trainable_params(model))
    
    #############################################################
    #############################################################
    #############################################################

    L = context_length  # 序列长度
    D = d_model         # 隐状态维度
    H = num_heads       # 注意力头数
    F = d_ff            # FFN中间维度
    V = vocab_size      # 词汇表大小
    d_k = D // H        # 每个注意力头的维度（D必须被H整除）

    # --------------------------
    # 1. 单个Transformer块的FLOPs
    # --------------------------
    per_block = {}

    # 1.1 注意力层的RMSNorm
    # 计算：元素平方（D*L） + 均值（D*L加法） + 缩放（D*L乘法+除法）
    rms_attn = 4 * D * L  # 简化近似：4次操作/元素（平方、加法、除法、乘法）

    # 1.2 多头自注意力
    # Q/K/V投影（3个线性层）：每个FLOPs=2*D*D*L（矩阵乘法）
    attn_qkv = 3 * 2 * D * D * L
    # RoPE旋转（Q和K各一次）：6*L*D（元素级乘加，之前推导）
    attn_rope = 6 * L * D
    # Q×K^T：2*H*L^2*d_k = 2*L^2*D（因H*d_k=D）
    attn_qk = 2 * (L **2) * D
    # 注意力权重×V：2*L^2*D（同Q×K^T）
    attn_v = 2 * (L** 2) * D
    # 输出投影（线性层）：2*D*D*L
    attn_out = 2 * D * D * L
    # 注意力总FLOPs
    attn_total = attn_qkv + attn_rope + attn_qk + attn_v + attn_out

    # 1.3 FFN层的RMSNorm（同注意力层的RMSNorm）
    rms_ffn = 4 * D * L

    # 1.4 SwiGLU前馈网络
    # W1和W3投影：2*D*F*L  each → 共4*D*F*L
    ffn_w1w3 = 4 * D * F * L
    # SiLU激活 + 逐元素相乘：2*F*L（每个元素2次操作）
    ffn_silu_mul = 2 * F * L
    # W2投影：2*F*D*L
    ffn_w2 = 2 * F * D * L
    # FFN总FLOPs
    ffn_total = ffn_w1w3 + ffn_silu_mul + ffn_w2

    # 单个块的总FLOPs
    per_block_total = rms_attn + attn_total + rms_ffn + ffn_total

    # --------------------------
    # 2. 所有层的总FLOPs
    # --------------------------
    total_layers = num_layers * per_block_total

    # --------------------------
    # 3. 输出层FLOPs
    # --------------------------
    # 最终RMSNorm
    output_rms = 4 * D * L
    # 输出线性投影（D→V）
    output_proj = 2 * D * V * L
    output_total = output_rms + output_proj

    # --------------------------
    # 总FLOPs
    # --------------------------
    total_flops = total_layers + output_total
    
    print("模型FLOPs分析：")
    print(f"  单块RMSNorm（注意力前）：{format_flops(rms_attn)}")
    print(f"  单块注意力总：{format_flops(attn_total)}")
    print(f"  单块RMSNorm（FFN前）：{format_flops(rms_ffn)}")
    print(f"  单块FFN总：{format_flops(ffn_total)}")
    print(f"  单块总FLOPs：{format_flops(per_block_total)}")
    print(f"  所有层总FLOPs：{format_flops(total_layers)}")
    print(f"  输出层总FLOPs：{format_flops(output_total)}")
    print(f"  模型总FLOPs：{format_flops(total_flops)}")

# 模型参数（用户提供）
params = {
    "vocab_size": 50257,
    "context_length": 16384,
    "num_layers": 48,
    "d_model": 1600,
    "num_heads": 25,
    "d_ff": 6400
}

# 计算FLOPs
calcu(** params)
    

2127057600
模型FLOPs分析：
  单块RMSNorm（注意力前）：104.86 MFLOPs
  单块注意力总：2.05 TFLOPs
  单块RMSNorm（FFN前）：104.86 MFLOPs
  单块FFN总：1.01 TFLOPs
  单块总FLOPs：3.06 TFLOPs
  所有层总FLOPs：146.92 TFLOPs
  输出层总FLOPs：2.64 TFLOPs
  模型总FLOPs：149.55 TFLOPs


# PART 4


- cross entropy
- learning_rate_tuning(TODO)
- adamw

### adamwAccounting
#####  a

- 参数内存 = 参数数量P * 4
- 激活内存（中间tensor）= B * L * d_model (Embedding 的)
                        \+ num_layer * (16 B * L * d_model + B * num_heads * L * L)
                        \+ B * L * vocab_size
- 梯度内存 = 4 * P
- 优化器状态 = 8 * P

- 总内存为4个加和

#####  b

固定内存 = 16P \sim 24.88 GB
激活内存 \sim 10.89 * B GB

算出 80GB Memory下 batch最大为5


#####  c

- forward的 FLOPs = 前面算的
- backward的 FLOPs = 2 * Forward_FLOPs = 2 * FP
- 优化器更新的 FLOPs = 4 * P
- 总共为 3 * FP + 4 * P

#####  d

- 总 FLOPs 计算
时间(天) = 总FLOPs / 有效硬件FLOP/s








- cosine learning rate schedule withwarmup

# PART 5

- save / load
- 